In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

happy_dir = "./data/happy/"
sad_dir = "./data/sad/"

print("Sample happy image: ")
plt.imshow(load_img(f"{os.path.join(happy_dir, os.listdir(happy_dir)[0])}"))
plt.show()

print("\nSample sad image: ")
plt.imshow(load_img(f"{os.path.join(sad_dir, os.listdir(sad_dir)[0])}"))
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array

sample_image = load_img(f"{os.path.join(happy_dir, os.listdir(happy_dir)[0])}")
sample_array = img_to_array(sample_image)

In [ ]:
class myCallBack(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get("accuracy") is not None and logs.get("accuracy") > 0.999:
            print("\n Reached 99.9% accuracy")
            self.model.stop_training = True

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def image_generator():

    train_datagen = ImageDataGenerator(rescale=1/255)

    train_generator = train_datagen.flow_from_directory(directory = './data/',
                                                        target_size=(150, 150),
                                                        batch_size=10,
                                                        class_mode='binary')
    return train_generator

In [ ]:
gen = image_generator()

In [ ]:
from tensorflow.keras import optimizer, losses

def train_happy_sad_model(train_generator):
    callbacks = myCallBack()

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation = 'relu', input_shape = (150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss=losses.BinaryCrossentropy() , optimizer=optimizer.RMSprop(learning_rate=0.001), metrics=["accuracy"])

    history = model.fit(x=train_generator, epochs=20, callbacks=callbacks)
    return history

In [ ]:
hist = train_happy_sad_model(gen)

print(f"Your model reaches the desired accuracy after {len(hist.epoch)} epochs")